### Cargado de datos

Este es el codigo con el que se van a cargar los datos, estos datos se van a cargar desde un JSON

In [2]:
import json

with open('../data/datos3.json') as file:
    data = json.load(file)

data

{'nodes': [{'nombre': 'Nodo1',
   'carrera': 10,
   'calle': 10,
   'semaforo': {'have': True, 'duration': 0.4},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4']},
  {'nombre': 'Nodo2',
   'carrera': 10,
   'calle': 20,
   'semaforo': {'have': True, 'duration': 1},
   'punto_de_interes': False,
   'conexiones': ['Nodo1', 'Nodo3']},
  {'nombre': 'Nodo3',
   'carrera': 10,
   'calle': 30,
   'semaforo': {'have': False},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo6']},
  {'nombre': 'Nodo4',
   'carrera': 20,
   'calle': 10,
   'semaforo': {'have': True, 'duration': 1.5},
   'punto_de_interes': False,
   'conexiones': ['Nodo5']},
  {'nombre': 'Nodo5',
   'carrera': 20,
   'calle': 20,
   'semaforo': {'have': False},
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4', 'Nodo6', 'Nodo8']},
  {'nombre': 'Nodo6',
   'carrera': 20,
   'calle': 30,
   'semaforo': {'have': True, 'duration': 0.9},
   'punto_de_interes': False,
   'conexiones': ['Nodo5'

### Objeto que se tiene que crear

El objeto Nodo o punto que se tiene que crear y con el metodo para cargar datos y crear un arreglo de objetos

In [3]:
class Nodo:
    def __init__(self, nombre, carrera, calle, semaforo, punto_de_interes, conexiones):
        self.nombre = nombre
        self.carrera = carrera
        self.calle = calle
        self.semaforo = semaforo
        self.punto_de_interes = punto_de_interes
        self.conexiones = conexiones
        self.g = 0
        self.h = 0
        self.f = 0
        self.parent = None

    def __str__(self):
        return (f"Nodo: {self.nombre}\n"
                f"Ubicación: Carrera {self.carrera}, Calle {self.calle}\n"
                f"Semaforo: {'Sí' if self.semaforo else 'No'}\n"
                f"Punto de Interés: {self.punto_de_interes}\n"
                f"Conexiones: {', '.join(con.nombre for con in self.conexiones)}\n"
                f"g: {self.g}, h: {self.h}, f: {self.f}\n"
                f"Parent: {self.parent.nombre if self.parent else 'Ninguno'}")

    """
    def __eq__(self, other):
        return self.nombre == other.nombre
    """
    def __hash__(self):
        return hash(self.nombre)


    def load_nodes(data):
        nodes = {}
        # Diccionario para mapear los nombres de nodos a objetos Nodo 
        node_map = {}
        for nodo_data in data["nodes"]:
            nodo = Nodo(
                nodo_data["nombre"],
                nodo_data["carrera"],
                nodo_data["calle"],
                nodo_data["semaforo"],
                nodo_data["punto_de_interes"],
                [],
            )
            nodes[nodo.nombre] = nodo
            # Agregamos el nodo al mapeo
            node_map[nodo.nombre] = nodo_data["conexiones"]
        # Se agregan las conexiones reales
        for k, v in nodes.items():
            v.conexiones = [nodes[conexion] for conexion in node_map[k]]
        return nodes

In [4]:
# Este seria el ejemplo de uso para saber si se cargaron bien los datos

# Arreglos de nodos (Ya nuestros datos cargados y convertidos en objetos de datos)
nodes_dict = Nodo.load_nodes(data=data)

# Para verificar que todo cargo bien hacemos un for para verificar dentro del array
for k, v in nodes_dict.items():
    print(f"{k}: {v.nombre}")
    # print([n.nombre for n in v.conexiones])

Nodo1: Nodo1
Nodo2: Nodo2
Nodo3: Nodo3
Nodo4: Nodo4
Nodo5: Nodo5
Nodo6: Nodo6
Nodo7: Nodo7
Nodo8: Nodo8
Nodo9: Nodo9


In [5]:
import heapq

def calculate_heuristic(node, end):
    return abs(node.calle - end.calle) + abs(node.carrera - end.carrera)

def calculate_heuristic_with_time(node, end):
    distancia_manhattan = abs(node.calle - end.calle) + abs(node.carrera - end.carrera)
    tiempo_transito_base = distancia_manhattan * 10

    tiempo_semaforo = 0

    if node.semaforo["have"]:
        tiempo_semaforo = node.semaforo['duration']

    return tiempo_transito_base + tiempo_semaforo

def astar(nodes_dict, start_name, end_name):
    open_list = []
    closed_list = set()

    start_node = nodes_dict[start_name]
    end_node = nodes_dict[end_name]

    heapq.heappush(open_list, ( start_node.nombre, start_node))

    while open_list:
        _, current_node = heapq.heappop(open_list)
        closed_list.add(current_node.nombre)

        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.nombre)
                current_node = current_node.parent
            return path[::-1]

        for neighbor in current_node.conexiones:
            if neighbor.nombre in closed_list:
                continue

            tentative_g = current_node.g + 1
            tentative_f = tentative_g + calculate_heuristic(neighbor, end_node)

            in_open_list = False
            for item in open_list:
                _, open_node = item
                if neighbor.nombre == open_node.nombre:
                    in_open_list = True
                    if tentative_g >= neighbor.g:
                        break
            if not in_open_list or tentative_g < neighbor.g:
                neighbor.g = tentative_g
                neighbor.h = calculate_heuristic(neighbor, end_node)
                neighbor.f = tentative_f
                neighbor.parent = current_node
                if not in_open_list:
                    heapq.heappush(open_list, ( neighbor.nombre, neighbor))

    return None


In [6]:
path = astar(nodes_dict=nodes_dict, start_name='Nodo7', end_name='Nodo1')
print(path)

['Nodo7', 'Nodo4', 'Nodo5', 'Nodo2', 'Nodo1']


In [7]:
import heapq


def calculate_cost(node, neighbor, vehicle_efficiency):
    distance = abs(node.calle - neighbor.calle) + abs(node.carrera - neighbor.carrera)

    return distance/vehicle_efficiency

def astar_with_gas(nodes_dict, start_name, end_name, vehicle_efficiency):
    open_list = []
    closed_list = set()

    start_node = nodes_dict[start_name]
    end_node = nodes_dict[end_name]

    heapq.heappush(open_list, (start_node.nombre, start_node))  # El costo inicial es 0

    while open_list:
        _, current_node = heapq.heappop(open_list)
        closed_list.add(current_node.nombre)

        if current_node == end_node:
            path = []
            total_fuel_consumption = current_node.g  # El consumo total de combustible
            while current_node:
                path.append(current_node.nombre)
                current_node = current_node.parent
            return path[::-1], total_fuel_consumption  # Devuelve también el consumo total de combustible

        for neighbor in current_node.conexiones:
            if neighbor.nombre in closed_list:
                continue

            tentative_g = current_node.g + calculate_cost(current_node, neighbor, vehicle_efficiency)
            tentative_f = tentative_g + calculate_heuristic(neighbor, end_node)

            in_open_list = False
            for item in open_list:
                _, open_node = item
                if neighbor.nombre == open_node.nombre:
                    in_open_list = True
                    if tentative_g >= neighbor.g:
                        break
            if not in_open_list or tentative_g < neighbor.g:
                neighbor.g = tentative_g
                neighbor.h = calculate_heuristic(neighbor, end_node)
                neighbor.f = tentative_f
                neighbor.parent = current_node
                if not in_open_list:
                    heapq.heappush(open_list, (neighbor.nombre, neighbor))

    return None

# Asumir un valor para 'vehicle_efficiency', por ejemplo, 15 km/L
vehicle_efficiency = 15

# Llamada a la función astar:
# path, total_fuel_consumption = astar(nodes_dict, "inicio", "fin", vehicle_efficiency)


In [8]:
# Reasignar a todos los nodos euristicas, f y g a 0

for k, v in nodes_dict.items():
    v.f = 0
    v.g = 0
    v.h = 0
print("Termine")

Termine


In [9]:
# Asumir un valor para 'vehicle_efficiency', por ejemplo, 15 km/L
vehicle_efficiency = 5.2

# Llamada a la función astar:
path, total_fuel_consumption = astar_with_gas(nodes_dict, "Nodo9", "Nodo1", vehicle_efficiency)
print(path)
print("El consumo de combustible fue: " + str(total_fuel_consumption))

for k, v in nodes_dict.items():
    v.f = 0
    v.g = 0
    v.h = 0
print("Termine")

['Nodo9', 'Nodo6', 'Nodo5', 'Nodo2', 'Nodo1']
El consumo de combustible fue: 7.692307692307692
Termine


In [10]:
from collections import deque
def bfs_find_path(start_node, target_node):
    visited = set()
    queue = deque()
    queue.append((start_node, [start_node]))

    while queue:
        current_node, path = queue.popleft()
        if current_node == target_node:
            return path

        for connection in current_node.conexiones:
            if connection not in visited:
                visited.add(connection)
                new_path = list(path)
                new_path.append(connection)
                queue.append((connection, new_path))

    return None

def path_tour_trip_best_first_search(start_node_name, nodos):
    start_node = nodos[start_node_name]
    points_of_interest = [node for node in nodos.values() if node.punto_de_interes and node != start_node]
    path = [start_node]
    current_node = start_node 

    while points_of_interest:
        closest_path = None
        for point in points_of_interest:
            temp_path = bfs_find_path(current_node, point)
            if temp_path and (closest_path is None or len(temp_path) < len(closest_path)):
                closest_path = temp_path

        if closest_path:
            # Avoid adding the current node twice if it's already at the end of the path
            if path[-1] == closest_path[0]:
                path.extend(closest_path[1:])
            else:
                path.extend(closest_path)
            current_node = closest_path[-1]
            points_of_interest.remove(current_node)
        else:
            break  # If no path is found to any of the remaining points, break out of the loop

    if path[-1] != start_node:
        return_path_to_start = bfs_find_path(current_node, start_node)
        if return_path_to_start:
            # Asegúrate de no duplicar el nodo actual en el path
            if return_path_to_start[0] == path[-1]:
                path.extend(return_path_to_start[1:])
            else:
                path.extend(return_path_to_start)

    return [node.nombre for node in path]


In [11]:
start_node = 'Nodo3'
camino_tour = path_tour_trip_best_first_search(start_node, nodes_dict)
print("Camino del toure: ", camino_tour)

Camino del toure:  ['Nodo3', 'Nodo2', 'Nodo1', 'Nodo4', 'Nodo5', 'Nodo8', 'Nodo7', 'Nodo8', 'Nodo9', 'Nodo6', 'Nodo5', 'Nodo2', 'Nodo3']


In [12]:
import math
nombre_nodos = list(nodes_dict.keys())

n = math.ceil(math.sqrt(len(nombre_nodos)))

matriz = []

for i in range(n):
    fila = []
    for j in range(n):
        # Calcular el índice del nodo en la lista plana
        index = i * n + j
        if index < len(nombre_nodos):
            fila.append(nombre_nodos[index])
        else:
            fila.append(None)  
    matriz.append(fila)

print(matriz)

[['Nodo1', 'Nodo2', 'Nodo3'], ['Nodo4', 'Nodo5', 'Nodo6'], ['Nodo7', 'Nodo8', 'Nodo9']]
